In [32]:
from nba_api.stats.endpoints import playercareerstats, playergamelog, playergamelogs
from nba_api.stats.static import players
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
def get_player_id_from_name(player_name):
    all_players = players.get_players()
    for player in all_players:
        if player['full_name'].lower() == player_name.lower():
            return player['id']
    return None

In [49]:
url = "https://www.nba.com/player/2343523652"
response = requests.get(url)
response.url

'https://www.nba.com/players'

In [53]:
def get_player_info(player_id):
    """
    Retrieve player information from the NBA website based on the provided player ID.

    Args:
        player_id (str): The unique identifier for the player.

    Returns:
        dict: A dictionary containing player information such as age, birth date, height, weight,
              country, draft information, last attended, and experience. If any information is not
              available, the corresponding value will be 'Unknown'.
    """

    url = f"https://www.nba.com/player/{player_id}"
    response = requests.get(url)
    player_info = {}
    label_value_map = {
        "AGE": "age",
        "BIRTHDATE": "birthdate",
        "HEIGHT": "height",
        "WEIGHT": "weight",
        "COUNTRY": "country",
        "DRAFT": "draft",
        "LAST ATTENDED": "last_attended",
        "EXPERIENCE": "experience"
        }

    # Scrape data from NBA page
    if response.url != 'https://www.nba.com/players' and response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        labels = soup.find_all('p', class_='PlayerSummary_playerInfoLabel__hb5fs')
        values = soup.find_all('p', class_='PlayerSummary_playerInfoValue__JS8_v')
        
        for label, value in zip(labels, values):
            label_text = label.text.strip().upper()  # Ensure case insensitivity
            value_text = value.text.strip()
            key = label_value_map.get(label_text)
            if key:
                player_info[key] = value_text
        return player_info
    
    # If response status is not 200, return default values
    default_value = 'Unknown'
    player_info = {key: default_value for key in label_value_map.values()}
    return player_info

# Example usage
player_name = "Lebron James"
player_id = get_player_id_from_name(player_name)
player_info = get_player_info(player_id)
player_info


{'height': '6\'9" (2.06m)',
 'weight': '250lb (113kg)',
 'country': 'USA',
 'last_attended': 'St. Vincent-St. Mary HS (OH)',
 'age': '39 years',
 'birthdate': 'December 30, 1984',
 'draft': '2003 R1 Pick 1',
 'experience': '20 Years'}

In [31]:
from nba_api.stats.endpoints import leaguedashplayerbiostats

player_name = "Michael Jordan"
player_id = get_player_id_from_name(player_name)
bruh = leaguedashplayerbiostats.LeagueDashPlayerBioStats().get_data_frames()[0]
bruh.loc[bruh['PLAYER_ID']==player_id]


,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,...,GP,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT


In [22]:
from nba_api.stats.endpoints import playerawards

player_name = "LeBron James"
player_id = get_player_id_from_name(player_name)
player_awards = playerawards.PlayerAwards(player_id=player_id).get_data_frames()[0]
player_awards.DESCRIPTION.unique()

array(['All-Defensive Team', 'All-NBA', 'All-Rookie Team',
       'NBA All-Star Most Valuable Player',
       'NBA Finals Most Valuable Player', 'NBA Most Valuable Player',
       'NBA Player of the Month', 'NBA Player of the Week',
       'NBA Rookie of the Month', 'NBA Rookie of the Year',
       'NBA Sporting News Most Valuable Player of the Year',
       'NBA Sporting News Rookie of the Year', 'Olympic Bronze Medal',
       'Olympic Gold Medal',
       'NBA In-Season Tournament Most Valuable Player',
       'NBA In-Season Tournament All-Tournament'], dtype=object)

In [20]:
# Set player name and retrieve player ID
player_name = "LeBron James"
player_id = get_player_id_from_name(player_name)

# Retrieve player career statistics
career = playercareerstats.PlayerCareerStats(player_id=player_id)
career_stats = career.get_data_frames()[0]
career_stats = career_stats.rename(columns={'TEAM_ABBREVIATION': 'TEAM', 'PLAYER_AGE': 'AGE'})

# Define column lists
totals_cols = ['MIN', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'FGM', 'FGA',
            'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
            'OREB', 'DREB', 'PF']
per_game_cols = ['MPG', 'PPG', 'RPG', 'APG', 'SPG', 'BPG', 'TPG', 'FGM', 'FGA',
                'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
                'ORPG', 'DRPG', 'PF']
percentage_stats = ['FG_PCT', 'FG3_PCT', 'FT_PCT']

# Career Totals
career_totals_per_season = career_stats[['SEASON_ID', 'AGE', 'GP', 'TEAM'] + totals_cols].copy()
career_totals_per_season['FULL_NAME'] = player_name

# Sum career totals and convert to int
career_totals = {'FULL_NAME': player_name, 'GP': int(career_totals_per_season['GP'].sum())}
career_totals.update(career_totals_per_season[totals_cols].sum().astype(int).to_dict())

# Convert non-integer columns to int except for 'FULL_NAME', 'SEASON_ID', and percentage columns
for col in career_totals_per_season.columns:
    if col not in ['FULL_NAME', 'SEASON_ID', 'TEAM'] + percentage_stats:
        career_totals_per_season[col] = career_totals_per_season[col].astype(int)

# Career Averages
career_avg_per_season = pd.DataFrame(columns=['FULL_NAME', 'SEASON_ID', 'AGE', 'TEAM', 'GP'] + per_game_cols)

# Calculate per game averages
for stat, per_game_stat in zip(totals_cols, per_game_cols):
    career_avg_per_season[per_game_stat] = round(career_stats[stat] / career_stats['GP'], 1)

career_avg_per_season['FULL_NAME'] = player_name
career_avg_per_season['SEASON_ID'] = career_stats['SEASON_ID']
career_avg_per_season['AGE'] = career_stats['AGE']
career_avg_per_season['TEAM'] = career_stats['TEAM']
career_avg_per_season['GP'] = career_stats['GP']
career_avg_per_season['AGE'] = career_avg_per_season['AGE'].astype('int')

# Calculate career averages
career_averages = {'FULL_NAME': player_name, 'GP': int(career_avg_per_season['GP'].sum())}
career_averages.update(round(career_avg_per_season[per_game_cols].mean(), 1).to_dict())

# Convert percentage Stats
for stat in percentage_stats:
    career_avg_per_season[stat] = round(career_stats[stat] * 100, 1)
    career_averages[stat] = round(career_avg_per_season[stat].mean(), 1)
    career_totals_per_season[stat] = round(career_totals_per_season[stat] * 100, 1)
    career_totals[stat] = round(career_avg_per_season[stat].mean(), 1)

# Convert DataFrames to dictionaries
career_totals_per_season_json = career_totals_per_season.to_dict(orient='records')
career_avg_per_season_json = career_avg_per_season.to_dict(orient='records')

career_totals_per_season

,SEASON_ID,AGE,GP,TEAM,MIN,PTS,REB,AST,STL,BLK,...,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,PF,FULL_NAME
0,2003-04,19,79,CLE,3120,1654,432,465,130,58,...,63,217,29.0,347,460,75.4,99,333,149,LeBron James
1,2004-05,20,80,CLE,3388,2175,588,577,177,52,...,108,308,35.1,477,636,75.0,111,477,146,LeBron James
2,2005-06,21,79,CLE,3361,2478,556,521,123,66,...,127,379,33.5,601,814,73.8,75,481,181,LeBron James
3,2006-07,22,78,CLE,3190,2132,526,470,125,55,...,99,310,31.9,489,701,69.8,83,443,171,LeBron James
4,2007-08,23,75,CLE,3027,2250,592,539,138,81,...,113,359,31.5,549,771,71.2,133,459,165,LeBron James
5,2008-09,24,81,CLE,3054,2304,613,587,137,93,...,132,384,34.4,594,762,78.0,106,507,139,LeBron James
6,2009-10,25,76,CLE,2966,2258,554,651,125,77,...,129,387,33.3,593,773,76.7,71,483,119,LeBron James
7,2010-11,26,79,MIA,3063,2111,590,554,124,50,...,92,279,33.0,503,663,75.9,80,510,163,LeBron James
8,2011-12,27,62,MIA,2326,1683,492,387,115,50,...,54,149,36.2,387,502,77.1,94,398,96,LeBron James
9,2012-13,28,76,MIA,2877,2036,610,551,129,67,...,103,254,40.6,403,535,75.3,97,513,110,LeBron James


In [16]:
5.538095238095238/7.55238095238095

0.7332912988650695

In [56]:
def get_player_image(player_name):
    """
    Gets NBA player headshot using player ID.

    Args:
        - player_name (str): Full name of current or former NBA player.

    Returns:
        - URL link to the headshot of NBA player.
    """

    player_id = get_player_id_from_name(player_name=player_name)
    url = f'https://cdn.nba.com/headshots/nba/latest/1040x760/{player_id}.png'

    return url

get_player_image("Jason Kidd")

'https://cdn.nba.com/headshots/nba/latest/1040x760/467.png'